
# Factchecking with LangGraph

In [5]:
!pip install langchain_community langchain_openai langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.6/437.6 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.56
    Uninstalling langchain-core-0.3.56:
      Successfully uninstalled langchain-core-0.3.56


In [6]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [7]:
#adding API KEYS for OpenAI and TAVILY
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

In [8]:
tool = TavilySearchResults(max_results=3) #increased number of results
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [9]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [10]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [25]:
prompt = """You are an expereinced FactChecker. Claim: "Mountain Rwenzori is the tallest Mountain in the World" \
Task: Fact-check this claim using current, reliable sources from the web. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
Instructions: 1. "Use a search tool to access up-to-date information" 2. "If the claim is verified by relaible sources, return: TRUE". \
3. "If the claim is contradicted by reliable sources, retur: FALSE" 4. If no conclusive information is found, return: PLAUSIBLE". \
Your Response:
[TRUE / FALSE / PLAUSIBLE]
Short explanation with source(s).
"""

model = ChatOpenAI(model="gpt-3.5-turbo")  #reduce inference cost
abot = Agent(model, [tool], system=prompt)

In [ ]:
#from IPython.display import Image
#Image(abot.graph.get_graph().draw_png())

In [20]:
messages = [HumanMessage(content="President Museveni is the President of Kenya")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'President Museveni President of Kenya'}, 'id': 'call_PTU08PiCwyIrST3Gvdjkvs05', 'type': 'tool_call'}
Back to the model!


In [21]:
result['messages'][-1].content

'FALSE\n\nPresident Museveni is not the President of Kenya. He is the President of Uganda. The search results confirm that President Museveni is the President of Uganda, not Kenya.'

In [26]:
messages = [HumanMessage(content="Uganda's literacy rate is higher than Kenya's and is at 90%")]
result = abot.graph.invoke({"messages": messages})
result['messages'][-1].content

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Uganda literacy rate vs Kenya literacy rate'}, 'id': 'call_Oa6JPaA9EWetMt7AOUDz9Qle', 'type': 'tool_call'}
Back to the model!


"The adult literacy rate in Kenya is 81.5%, while in Uganda, it is 76.5%. Therefore, the claim that Uganda's literacy rate is higher than Kenya's at 90% is false.\n\nSources:\n1. [Versus - Adult literacy rate: how does Kenya compare to Uganda?](https://versus.com/en/kenya-vs-uganda/literacy-rate)\n2. [East African Community Data Portal - Literacy Rate](https://eac.opendataforafrica.org/krosppc/literacy-rate)\n3. [NationMaster - Kenya vs Uganda Education Stats Compared](https://www.nationmaster.com/country-info/compare/Kenya/Uganda/Education)"